In [12]:
import pandas as pd

In [13]:
df_train = pd.read_csv('train_IxoE5JN.csv')
df_test = pd.read_csv('test_WudNWDM.csv')
df_submission = pd.read_csv('sample_submission_jn0a7vR.csv')

In [14]:
df_submission.head(2)

,row_id,energy
0,94993,1702.995014
1,94994,1702.995014


In [15]:
df_train.head(2)

,row_id,datetime,energy
0,1,2008-03-01 00:00:00,1259.985563
1,2,2008-03-01 01:00:00,1095.541500


In [16]:
df_test.head(2)

,row_id,datetime
0,94993,2019-01-01 00:00:00
1,94994,2019-01-01 01:00:00


In [17]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94992 entries, 0 to 94991
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   row_id    94992 non-null  int64  
 1   datetime  94992 non-null  object 
 2   energy    93092 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


In [18]:
df_train.isna().sum()

row_id         0
datetime       0
energy      1900
dtype: int64

In [19]:
df_train['energy'].fillna(df_train['energy'].mean(),inplace=True)

In [20]:
df_train.isna().sum()

row_id      0
datetime    0
energy      0
dtype: int64

In [21]:
new_df_train = df_train
new_df_train['Day'] = pd.to_datetime(new_df_train['datetime']).dt.dayofweek
new_df_train['Hour'] = pd.to_datetime(new_df_train['datetime']).dt.hour
new_df_train['Month'] = pd.to_datetime(new_df_train['datetime']).dt.month
new_df_train['Year'] = pd.to_datetime(new_df_train['datetime']).dt.year
new_df_train['Quarter'] = pd.to_datetime(new_df_train['datetime']).dt.quarter

new_df_train = df_train.set_index('datetime')
new_df_train.index = pd.to_datetime(new_df_train.index)

new_df_test = df_test
new_df_test['Day'] = pd.to_datetime(new_df_test['datetime']).dt.dayofweek
new_df_test['Hour'] = pd.to_datetime(new_df_test['datetime']).dt.hour
new_df_test['Month'] = pd.to_datetime(new_df_test['datetime']).dt.month
new_df_test['Year'] = pd.to_datetime(new_df_test['datetime']).dt.year
new_df_test['Quarter'] = pd.to_datetime(new_df_test['datetime']).dt.quarter

new_df_test = df_test.set_index('datetime')
new_df_test.index = pd.to_datetime(new_df_test.index)

In [23]:
new_df_test.head(2)
new_df_train.isna().sum()

row_id     0
energy     0
Day        0
Hour       0
Month      0
Year       0
Quarter    0
dtype: int64

In [64]:
import xgboost as xgb

In [65]:
from sklearn.metrics import mean_squared_error

In [25]:
x_train = new_df_train.drop(['energy'],axis=1)
y_train = new_df_train['energy']


In [26]:
new_df_train.isna().sum()

row_id     0
energy     0
Day        0
Hour       0
Month      0
Year       0
Quarter    0
dtype: int64

In [27]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [28]:
lr.fit(x_train,y_train)

LinearRegression()

In [30]:
predict = lr.predict(new_df_test)

In [33]:
idx = new_df_test['row_id']

In [36]:
data = {'row_id':idx,'energy':predict}

In [44]:
dat = pd.DataFrame(data)
dat.to_csv('submission.csv',index=False)

In [43]:
df_submission

,row_id,energy
0,94993,1702.995014
1,94994,1702.995014
2,94995,1702.995014
3,94996,1702.995014
4,94997,1702.995014
...,...,...
26299,121292,1702.995014
26300,121293,1702.995014
26301,121294,1702.995014
26302,121295,1702.995014


In [45]:
final_df = pd.read_csv('submission.csv')
final_df

,row_id,energy
0,94993,1882.660475
1,94994,1899.405037
2,94995,1916.149598
3,94996,1932.894159
4,94997,1949.638721
...,...,...
26299,121292,2428.888693
26300,121293,2445.633254
26301,121294,2462.377815
26302,121295,2479.122376


In [47]:
from sklearn.ensemble import RandomForestRegressor

In [59]:
rf = RandomForestRegressor(n_estimators = 50,max_leaf_nodes=50,max_depth=100)

In [60]:
rf.fit(x_train,y_train)

RandomForestRegressor(max_depth=100, max_leaf_nodes=50, n_estimators=50)

In [62]:
predict = lr.predict(new_df_test)
idx = new_df_test['row_id']
data = {'row_id':idx,'energy':predict}
dat = pd.DataFrame(data)
dat.to_csv('submission3.csv',index=False)
final_df = pd.read_csv('submission3.csv')
final_df

,row_id,energy
0,94993,1882.660475
1,94994,1899.405037
2,94995,1916.149598
3,94996,1932.894159
4,94997,1949.638721
...,...,...
26299,121292,2428.888693
26300,121293,2445.633254
26301,121294,2462.377815
26302,121295,2479.122376


In [99]:
xg = xgb.XGBRegressor(n_estimators = 8, seed = 123,eta=0.4,max_depth=15)

In [100]:
xg.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False, eta=0.4,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.400000006, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=15, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=8, n_jobs=0,
             num_parallel_tree=1, predictor='auto', ...)

In [101]:
predict = xg.predict(new_df_test)
idx = new_df_test['row_id']
data = {'row_id':idx,'energy':predict}
dat = pd.DataFrame(data)
dat.to_csv('submission4.csv',index=False)
final_df = pd.read_csv('submission4.csv')
final_df

,row_id,energy
0,94993,1680.9462
1,94994,1614.4823
2,94995,1578.9349
3,94996,1510.0472
4,94997,1501.9710
...,...,...
26299,121292,2723.5261
26300,121293,2519.6624
26301,121294,2334.2236
26302,121295,2334.2236
